# Create a Model Inference Nuclio Function

- [Initialize Nuclio Emulation, Environment Variables, and Configuration](#netops-infer-init-func)
  - [Install Required Packages](#netops-infer-install-pkgs)
- [Set a cron Function Trigger](#netops-infer-set-func-trigger)
- [Initialize the Function](#netops-infer-func-init)
- [Deploy the Function](#netops-infer-func-deploy)

<a id="netops-infer-init-func"></a>
## Initialize Nuclio Emulation, Environment Variables, and Configuration

> **Note:** Use `# nuclio: ignore` for sections that dont need to be copied to the function.

In [1]:
# nuclio: ignore
import nuclio

In [2]:
# nuclio: ignore
from os import environ
version = '1.0'
environ['MODEL_FILE'] = 'netops.v{}.model'.format(version)

In [3]:
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c MODEL_FILE=${MODEL_FILE}
%nuclio env -c MODEL_FILEPATH=/tmp/mlmodel/${MODEL_FILE}
%nuclio env -l MODEL_FILEPATH=models/${MODEL_FILE}

%nuclio: setting 'MODEL_FILEPATH' environment variable


<a id="netops-infer-install-pkgs"></a>
### Install Required Packages

`%nuclio cmd` allows you to run image build instructions and install packages.

> **Note:** When using the `-c` option, commands will only run in Nuclio and not locally.

In [4]:
%%nuclio cmd -c
pip install pandas
pip install sklearn
pip install v3io_frames
pip install dask_ml
apt-get update && apt-get install -y wget
mkdir -p /tmp/mlmodel

In [5]:
# Copy the model file into the function
%nuclio cmd -c wget -O /tmp/mlmodel/${MODEL_FILE} --header "x-v3io-session-key: ${V3IO_ACCESS_KEY}" http://${V3IO_WEBAPI_SERVICE_HOST}:8081/users/${V3IO_USERNAME}/demos/netops/models/${MODEL_FILE}

<a id="netops-infer-set-func-trigger"></a>
## Set a cron Function Trigger

Set a cron function trigger with 1-minute interval:

In [6]:
%%nuclio config 
spec.triggers.secs.kind = "cron"
spec.triggers.secs.attributes.interval = "600s"
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.secs.kind to 'cron'
%nuclio: setting spec.triggers.secs.attributes.interval to '600s'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


<a id="netops-infer-func-init"></a>
## Initialize the Function

In [7]:
import pandas as pd
import pickle
import sklearn
from dask_ml.wrappers import ParallelPostFit
import v3io_frames as v3f
from os import environ

# Define a V3IO TSDB client
client = v3f.Client('framesd:8081')

# Load the model
with open(environ['MODEL_FILEPATH'], 'rb+') as f:
    model = pickle.load(f) 
    
clf = ParallelPostFit(estimator=model,
                      scoring='accuracy')

In [8]:
%%nuclio handler
# Load last hour data
df = client.read(backend='tsdb', query='select avg(cpu_utilization) , avg(latency) , avg(packet_loss) , avg(throughput) from netops_metrics_jupyter group by company, data_center, device',
            start="now-1d", end='now+1d', multi_index=True, step='1h')
df['prediction'] = clf.predict(df)
#df.reset_index(inplace=True)
#client.write(backend='tsdb', table='netops_predictions', dfs=[df], index_cols=['company', 'data_center', 'device', 'time'])
str(df['prediction']) # nuclio: return

'time                 company                     data_center        device       \n2019-06-17 13:42:04  Ortega__Nelson_and_Parrish  Chen_Rest          9999032632997    0.0\n2019-06-17 14:42:04  Ortega__Nelson_and_Parrish  Chen_Rest          9999032632997    0.0\n2019-06-17 13:42:04  Ortega__Nelson_and_Parrish  Chen_Rest          2588823083436    0.0\n2019-06-17 14:42:04  Ortega__Nelson_and_Parrish  Chen_Rest          2588823083436    0.0\n2019-06-17 13:42:04  Ortega__Nelson_and_Parrish  Gallagher_Springs  9909218583192    0.0\n2019-06-17 14:42:04  Ortega__Nelson_and_Parrish  Gallagher_Springs  9909218583192    0.0\n2019-06-17 13:42:04  Washington-Lambert          Tamara_Greens      9025508674926    0.0\n2019-06-17 14:42:04  Washington-Lambert          Tamara_Greens      9025508674926    0.0\n2019-06-17 13:42:04  Washington-Lambert          Nicole_Trail       5079628775825    0.0\n2019-06-17 14:42:04  Washington-Lambert          Nicole_Trail       5079628775825    0.0\n2019-06-17 13:42

<a id="netops-infer-func-deploy"></a>
## Deploy the Function

In [ ]:
%nuclio deploy -n netops-infer -p netops -c

In [10]:
client.read(backend='tsdb', query='select avg(cpu_utilization) , avg(latency) , avg(packet_loss) , avg(throughput)  from netops_metrics_jupyter group by company, data_center, device',
            start="now-1d", end='now+1d', multi_index=True, step='5m').head(10)


,,,,avg(cpu_utilization),avg(latency),avg(packet_loss),avg(throughput)
time,company,data_center,device,,,,
2019-06-17 14:38:29,Washington-Lambert,Tamara_Greens,1416256267840,77.711198,30.534627,9.209405,233.679544
2019-06-17 14:43:29,Washington-Lambert,Tamara_Greens,1416256267840,79.533761,29.101493,14.341944,179.910993
2019-06-17 14:48:29,Washington-Lambert,Tamara_Greens,1416256267840,70.554952,1.738742,0.911834,250.449321
2019-06-17 14:53:29,Washington-Lambert,Tamara_Greens,1416256267840,69.348307,1.350994,0.524656,249.755806
2019-06-17 14:58:29,Washington-Lambert,Tamara_Greens,1416256267840,68.872050,2.495677,0.712196,252.960565
2019-06-17 15:03:29,Washington-Lambert,Tamara_Greens,1416256267840,80.429847,31.620278,14.888336,182.140072
2019-06-17 15:08:29,Washington-Lambert,Tamara_Greens,1416256267840,76.459967,25.483856,12.512834,201.100587
2019-06-17 15:13:29,Washington-Lambert,Tamara_Greens,1416256267840,85.523864,45.786581,21.547878,149.405401
2019-06-17 15:18:29,Washington-Lambert,Tamara_Greens,1416256267840,84.407529,48.858549,20.431261,158.104648
